<br><br><br>
<h2><font color="#004D7F" size=6>Práctica 3</font></h2>



<h1><font color="#004D7F" size=5> SISTEMAS BASADOS EN REGLAS DIFUSAS. ALGORITMO DE WANG Y MENDEL.</font></h1>

<br>
<div style="text-align: right">
<font color="#004D7F" size=3>Guillermo Tomás Fernández Martin y José Miguel Puerta Callejón</font><br>
<font color="#004D7F" size=3>Guillermo.Fernandez@uclm.es</font><br>
<font color="#004D7F" size=3>Sistemas Basados en el Conocimiento</font><br>
<font color="#004D7F" size=3>Grado en Ingeniería Informática (UCLM)</font><br>
</div>


<div style="text-align: left">
<font color="#004D7F" size=4>ESTUDIANTES: </font><br>
<font color="#004D7F" size=4>PABLO LARIO GOMEZ</font><br>
<font color="#004D7F" size=4>DIEGO MIGUEL LOPEZ GARCIA</font><br>
<font color="#004D7F" size=3>Nombre de archivo lluvias Australia: 'weatherAUS.csv', directorio: mismo que la libreta</font><br>
</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [1. Introducción](#introduccion)
* [2. Metodología](#meto)
* [3. Implementación](#scikit)
* [4. Resultados](#c45)
* [5. Entrega](#entrega)


---

<br>

<a id="introduccion"></a>
# <font color="#004D7F"> 1. Introducción</font>

En esta práctica vamos a partir del Sistema Basado en Reglas implementado para la práctica anterior y vamos a mantener el motor de inferencia, pero a la hora de generar las reglas se realizará mediante el algoritmo Wang y Mendel. Ésto significa que ya no será necesario el árbol de clasificación de la práctica 1 para generar el sistema basado en reglas.



---

<br>

<a id="data"></a>
# <font color="#004D7F"> 2. Metodología</font>



### <font color="#004D7F"> 2.1 Generación de los conjuntos difusos</font>

Al contrario que en la práctica anterior, en esta ocasión no es necesario fuzzificar la base de datos. Sí es necesario, sin embargo, generar las variables difusas a partir de la base de datos, ya que se utilizarán para sacar el grado de pertenencia para cada posible regla. Para ello, se puede utilizar el método `fit` del `FuzzyEncoder` ya creado.

### <font color="#004D7F"> 2.2 Extracción de las reglas</font>

En este caso, se extraerá una regla por cada uno de las instancias existentes en la base de datos. A partir de esa regla, se comprobará la base de datos y se introducirá o no en el sistema basado en reglas. 

### <font color="#004D7F"> 2.3 Motor de inferencia</font>

Una vez extraídas las reglas, se usará el código utilizado en la práctica anterior para realizar la inferencia de una nueva instancia en el sistema basado en reglas.

---

<br>

<a id="data"></a>
# <font color="#004D7F"> 3. Implementación</font>

Para esta práctica el alumno debe llevar a cabo la implementación de las tres etapas descritas en la metodología, con el objetivo final de crear un sistema de reglas difusas basado en Wang y Mendel para el problema de la predicción de la lluvia en Australia y ser capaz de realizar predicciones a partir de la inferencia.

Vamos a describir un poco más en detalle cada uno de estos pasos:



### <font color="#004D7F"> 3.1 Variables Difusas</font>

El primer paso será generar las variables difusas a partir de las variables de la base de datos. Para ello, se proporciona el esqueleto de una clase que os ayudará a estructurar las variables con los elementos necesarios para la transformación posterior de la base de datos. Para simplificar, se utilizarán conjuntos difusos triangulares.

Los atributos de la clase serán los siguientes:
- `universe`: El universo del discurso de la variable. **IMPORTANTE** `scikit-fuzzy` asume que el universo del discurso está ordenado para todas sus operaciones. Sin embargo, cuando extraemos una columna de la base de datos para generar la variable difusa no tiene por qué estar ordenada. Habrá que ordenarla para poder utilizar las funciones de `scikit-fuzzy` pero sin modificar la base de datos original.
- `memb_func`: Un diccionario con los grados de pertenencia de cada elemento del universo del discurso para cada uno de los conjuntos difusos que se generen de la variable difusa
- `labels`: Etiquetas de cada uno de los conjuntos difusos
- `name`: Nombre de la variable difusa

Las funciones de la clase serán las siguientes:
- `_membership_functions`: Función interna para generar el diccionario con las distintas funciones de pertenencia. A COMPLETAR
- `fuzzify`: Función que devuelve el mejor conjunto difuso para un valor dentro del universo del discurso. A COMPLETAR
- `plot`: Función que muestra de manera gráfica los conjuntos difusos de la variable. Ya viene dada.

Para utilizar la clase, habrá que calcular previamente los triángulos de los conjuntos difusos. Para ello, se utilizará la estructura `fuzzy_points`, que es diccionario con la definición de los triángulos y forma `{label: [start, peak, end]}` para cada triángulo.

Para generar esta estructura `fuzzy_points` se recomienda calcular las particiones de las variables de manera automática, ya sea calculando por anchura o por frecuencia. `Pandas` tiene funciones que podéis buscar para discretizar una variable tanto por anchura como por frecuencia, se recomienda utilizarlas.

In [76]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import math

class FuzzyVariable:
    def __init__(self, name, variable, fuzzy_points):
        self.universe = np.sort(df[variable].to_numpy())
        self.labels = ['Low', 'Mid', 'High']
        self.memb_func = self._membership_functions(variable, fuzzy_points)
        self.name = name
        self.fuzzyPoints = fuzzy_points
    
    def _membership_functions(self, variable, fuzzy_points):
        fuzz_dict = {}

        # GENERAR LOS DICCIONARIOS CON LAS FUNCIONES DE PERTENENCIA
        aux = np.arange(len(self.universe))
        for i in range(len(aux)):
            aux[i] = 0

        fuzz_dict['Low'] = aux
        fuzz_dict['Mid'] = aux
        fuzz_dict['High'] = aux

        # fuzz_dict['Low'] = fuzz.trimf(self.universe, fuzzy_points['Low'])
        # fuzz_dict['Mid'] = fuzz.trimf(self.universe, fuzzy_points['Mid'])
        # fuzz_dict['High'] = fuzz.trimf(self.universe, fuzzy_points['High'])

        return fuzz_dict
    
    def fuzzify(self, value):
        best = ''
        max_val = -1

        # for label in self.labels: # Iterar por cada variable para ver la mejor
        #     # DEVOLVER EL CONJUNTO DIFUSO QUE MAYOR GRADO DE PERTENENCIA TENGA
        #     pass

        p1 = self.fuzzyPoints["Low"][1]
        p2 = self.fuzzyPoints["Mid"][1]
        p3 = self.fuzzyPoints["High"][1]
        
        best = fuzzificar(value, p1, p2, p3)

        return best

    def plot(self):
        fig, ax0 = plt.subplots(nrows=1)
        for label in self.labels:
            ax0.plot(self.universe, self.fuzz_dict[label], linewidth=1.5, label=label)
        ax0.set_title(self.name)
        ax0.legend()
        plt.tight_layout()

In [77]:
def fuzzificar(value, p1, p2, p3):

    a = (p1 + p2) / 2
    b = (p2 + p3) / 2

    if value < a:
        return "Low"
    elif value < b:
        return "Mid"
    else:
        return "High"

In [78]:
from sklearn.base import BaseEstimator


# Ejemplo de la estructura básica de un algoritmo que hereda de BaseEstimator siguiendo la estructura de SciKit
class FuzzyEncoder(BaseEstimator):
    # constructor
    def __init__(self, sets=3, labels=['Low', 'Mid', 'High']):
        self.fuzzy_vars = {}
        self.sets = sets
        self.labels = labels
        self.aux_fuzzy_vars = {}

    # siguiendo las guias de scikit-learn, creamos las funciones fit para el entrenamiento
    def fit(self, X):
        # APRENDER LAS VARIABLES DIFUSAS A PARTIR DE LOS DATOS

        for i in range(len(X.columns)):

            columna = X.columns[i:i+1]
            array = X[columna].to_numpy()

            arrayUnidimensional = np.reshape(array, (1, np.product(array.shape)))[0] # Sacar un array unidimensional

            if (type(arrayUnidimensional[0]) == str):
                continue

            equal_width_points = self._equal_width(arrayUnidimensional, 3)  # Mismo que en el ejemplo

            fuzzy_points = self._expand_points(equal_width_points, ['Low', 'Mid', 'High'])  # Mismo que en el ejemplo

            f = FuzzyVariable(columna[0], columna[0], fuzzy_points)
            self.aux_fuzzy_vars[columna[0]] = {
                'universe' : f.universe,
                'pert': {l: f.memb_func[l] for l in f.labels}
            }

            self.fuzzy_vars[columna[0]] = equal_width_points

        return self
        
    # y transform para aplicar lo aprendido y transformar la base de datos creando una nueva
    def transform(self, X):
        new_X = X.copy()

        # MODIFICAR new_X CON LAS VARIABLES DIFUSAS APRENDIDAS

        for columna in self.fuzzy_vars:
            
            equal_width_points = self.fuzzy_vars[columna]
            
            new_X[columna] = X[columna].apply(fuzzificar, args=equal_width_points)

        return new_X
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)

    def _equal_width(self, var, sets):
        try:
            cut = pd.cut(var, sets - 1)
            sol = [cat.left for cat in cut.categories] + [cut.categories[-1].right]
        except ValueError: 
            cut = pd.cut(var, sets - 1, duplicates='drop')
            sol = [var.min()] + [cat.left for cat in cut.categories] + [cut.categories[-1].right]
        sol[0] = var.min()
        return sol

    def _expand_points(self, fp, labels):
        ep = {}
        expanded_points = [fp[0]] + fp + [fp[-1]]

        for i in range(len(expanded_points) - 2):
            ep[labels[i]] = [expanded_points[i], expanded_points[i+1], expanded_points[i+2]]

        return ep

### <font color="#004D7F"> 3.2 Preprocesamiento</font>


Se ha de realizar el mismo preprocesamiento que en la práctica 1 con dos nuevas restricciones:
- La variable `Location` hay que eliminarla. Esto lo vamos a hacer para que el árbol que salga no ramifique demasiado
- Las variables de puntos cardinales hay que simplificarlas. Se han de quedar simplemente con los cuatro puntos cardinales principales (N, S, E, W)

In [79]:
# Preprocesamiento de la BBDD de la practica 1

df = pd.read_csv('weatherAUS.csv')

# Eliminamos todos los valores nulos

# Variables continuas
df['MinTemp'] = df['MinTemp'].fillna(df['MinTemp'].mean())
df['MaxTemp'] = df['MaxTemp'].fillna(df['MaxTemp'].mean())
df['Rainfall'] = df['Rainfall'].fillna(df['Rainfall'].mean())
df['WindGustSpeed'] = df['WindGustSpeed'].fillna(df['WindGustSpeed'].mean())
df['WindSpeed9am'] = df['WindSpeed9am'].fillna(df['WindSpeed9am'].mean())
df['WindSpeed3pm'] = df['WindSpeed3pm'].fillna(df['WindSpeed3pm'].mean())
df['Humidity9am'] = df['Humidity9am'].fillna(df['Humidity9am'].mean())
df['Humidity3pm'] = df['Humidity3pm'].fillna(df['Humidity3pm'].mean())
df['Pressure9am'] = df['Pressure9am'].fillna(df['Pressure9am'].mean())
df['Pressure3pm'] = df['Pressure3pm'].fillna(df['Pressure3pm'].mean())
df['Cloud9am'] = df['Cloud9am'].fillna(df['Cloud9am'].mean())
df['Cloud3pm'] = df['Cloud3pm'].fillna(df['Cloud3pm'].mean())
df['Temp9am'] = df['Temp9am'].fillna(df['Temp9am'].mean())
df['Temp3pm'] = df['Temp3pm'].fillna(df['Temp3pm'].mean())
df['Evaporation'].replace(np.nan, df['Evaporation'].mean(), inplace=True)
df['Sunshine'].replace(np.nan, df['Sunshine'].mean(), inplace=True)

# Variables categóricas
df['WindGustDir'].replace(np.nan, 'W', inplace=True)
df['WindDir9am'].replace(np.nan, 'N', inplace=True)
df['WindDir3pm'].replace(np.nan, 'SE', inplace=True)
df['RainToday'].replace(np.nan, 'No', inplace=True)
df['RainTomorrow'].replace(np.nan, 'No', inplace=True)

# Simplificamos en solo los cuatro puntos cardinales
df['WindGustDir'].replace('WNW', 'W', inplace=True)
df['WindGustDir'].replace('WSW', 'W', inplace=True)
df['WindGustDir'].replace('NE', 'N', inplace=True)
df['WindGustDir'].replace('NNW', 'N', inplace=True)
df['WindGustDir'].replace('NNE', 'N', inplace=True)
df['WindGustDir'].replace('SSE', 'S', inplace=True)
df['WindGustDir'].replace('NW', 'N', inplace=True)
df['WindGustDir'].replace('SW', 'S', inplace=True)
df['WindGustDir'].replace('ENE', 'E', inplace=True)
df['WindGustDir'].replace('SE', 'S', inplace=True)
df['WindGustDir'].replace('ESE', 'E', inplace=True)
df['WindGustDir'].replace('SSW', 'S', inplace=True)

df['WindDir9am'].replace('NNW', 'N', inplace=True)
df['WindDir9am'].replace('SE', 'S', inplace=True)
df['WindDir9am'].replace('ENE', 'E', inplace=True)
df['WindDir9am'].replace('SW', 'S', inplace=True)
df['WindDir9am'].replace('SSE', 'S', inplace=True)
df['WindDir9am'].replace('NE', 'N', inplace=True)
df['WindDir9am'].replace('SSW', 'S', inplace=True)
df['WindDir9am'].replace('WSW', 'W', inplace=True)
df['WindDir9am'].replace('ESE', 'E', inplace=True)
df['WindDir9am'].replace('NW', 'N', inplace=True)
df['WindDir9am'].replace('WNW', 'W', inplace=True)
df['WindDir9am'].replace('NNE', 'N', inplace=True)

df['WindDir3pm'].replace('WNW', 'W', inplace=True)
df['WindDir3pm'].replace('WSW', 'W', inplace=True)
df['WindDir3pm'].replace('NW', 'N', inplace=True)
df['WindDir3pm'].replace('SSE', 'S', inplace=True)
df['WindDir3pm'].replace('ESE', 'E', inplace=True)
df['WindDir3pm'].replace('ENE', 'E', inplace=True)
df['WindDir3pm'].replace('NNW', 'N', inplace=True)
df['WindDir3pm'].replace('SSW', 'S', inplace=True)
df['WindDir3pm'].replace('SW', 'S', inplace=True)
df['WindDir3pm'].replace('SE', 'S', inplace=True)
df['WindDir3pm'].replace('NNE', 'N', inplace=True)
df['WindDir3pm'].replace('NE', 'N', inplace=True)

features = df.columns
attributes = features[2:-1] # La variable Location queda eliminada
target = features[-1]


X = df[attributes]
y = df[target]
y[y == 'No'] = 0  # Los sistemas de scikit-fuzzy necesitan que el consecuente sea un numero en lugar de una etiqueta
y[y == 'Yes'] = 1
y = y.astype('float')

discrete = []
for columna in X:
    if columna == 'RainTomorrow':
        continue
    if isinstance(X[columna][0], str):
        discrete.append(columna)

C:\Users\Diego\AppData\Local\Temp\ipykernel_6500\95635843.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == 'No'] = 0  # Los sistemas de scikit-fuzzy necesitan que el consecuente sea un numero en lugar de una etiqueta
C:\Users\Diego\AppData\Local\Temp\ipykernel_6500\95635843.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == 'Yes'] = 1


In [80]:
fuzzy_vars = FuzzyEncoder()
fuzzy_X = fuzzy_vars.fit(X).transform(X)
fuzzy_X

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,Mid,Mid,Low,Low,Mid,W,Mid,W,W,Low,Mid,Mid,Low,Mid,Mid,High,Mid,Mid,Mid,No
1,Mid,Mid,Low,Low,Mid,W,Mid,N,W,Low,Mid,Mid,Mid,Mid,Mid,Mid,Mid,Mid,Mid,No
2,Mid,Mid,Low,Low,Mid,W,Mid,W,W,Low,Mid,Mid,Mid,Mid,Mid,Mid,Low,Mid,Mid,No
3,Mid,Mid,Low,Low,Mid,N,Low,S,E,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
4,Mid,Mid,Low,Low,Mid,W,Mid,E,N,Low,Low,High,Mid,Mid,Mid,High,High,Mid,Mid,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,Mid,Mid,Low,Low,Mid,E,Low,S,E,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145456,Mid,Mid,Low,Low,Mid,N,Low,S,N,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145457,Mid,Mid,Low,Low,Mid,N,Low,S,W,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145458,Mid,Mid,Low,Low,Mid,S,Low,S,N,Low,Low,Mid,Low,Mid,Mid,Mid,Low,Mid,Mid,No


Vamos a utilizar `scikit-fuzzy` para el sistema de control. Eso significa que se necesita un procesado previo para las variables discretas. Se necesitan dos elementos para crear un sistema basado en reglas con `scikit-fuzzy`:
- Universo del discurso. Para una variable discreta, será una enumeración de sus valores. Esto quiere decir que una variable `['N', 'S', 'E', 'W']` tendrá como universo del discurso `[0,1,2,3]`
- Una función de grados de pertenencia. En lugar de utilizar `fuzz.trapmf` o `fuzz.trimf`, la función será `1` cuando coincida con el elemento y `0` cuando no coincida. Para la variable anterior, 
  - `pert['N'] = [1,0,0,0]`,   
  - `pert['E'] = [0,0,1,0]`, 
  - etc.

Se recomienda crear un diccionario para cada una de las variables discretas para luego la generación de las reglas

In [81]:
def get_discrete(X, var):

    universeArray = None

    if var == 'RainTomorrow':
        universeArray = X.unique()
    else:
        universeArray = X[var].unique()

    # Creamos el universo de la variable
    universe = np.array(universeArray)
    universe = np.array(range(len(universe)))

    pert = {}
    i = 0
    for uniqueValue in universeArray:

        array = np.array(range(len(universe)))
        array = np.zeros_like(array)

        # Creamos la función de pertenencia de la variable
        pert[uniqueValue] = array
        pert[uniqueValue][i] = 1
        i += 1

    return {
        'universe': universe,
        'pert': pert
    }

Con este generador, dada la función `get_discrete` que nos genera las variables discretas, se consigue un diccionario por cada una de las variables discretas

In [82]:
# Calculamos las variables discretas
discrete_variables = { att: get_discrete(X, att) for att in discrete }
discrete_variables

{'WindGustDir': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'N': array([0, 1, 0, 0]),
   'S': array([0, 0, 1, 0]),
   'E': array([0, 0, 0, 1])}},
 'WindDir9am': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'N': array([0, 1, 0, 0]),
   'S': array([0, 0, 1, 0]),
   'E': array([0, 0, 0, 1])}},
 'WindDir3pm': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'E': array([0, 1, 0, 0]),
   'N': array([0, 0, 1, 0]),
   'S': array([0, 0, 0, 1])}},
 'RainToday': {'universe': array([0, 1]),
  'pert': {'No': array([1, 0]), 'Yes': array([0, 1])}}}

De igual manera, hay que crear un diccionario con la variable clase, que también es discreta. Se crea igual que el resto, pero se aconseja pasar esta variable por separado. 

In [83]:
# Calculamos la variable objetivo
target_variable = {}
target_variable['RainTomorrow'] = get_discrete(y, 'RainTomorrow')
target_variable

{'RainTomorrow': {'universe': array([0, 1]),
  'pert': {0.0: array([1, 0]), 1.0: array([0, 1])}}}

Finalmente se crea el sistema basado en reglas difusas. Vamos a tener una clase abstracta general y una clase que extienda de esta y utilice Wang y Mendel para obtener las reglas.

La clase abstracta `RuleBasedSystem`, igual que en la práctica anterior, se dividirá en las funciones:
- `fit`: Metodo abstracto que se programará en la clase que extienda de `RuleBasedSystem`
- `_create_rbs`: encargado de aprender el sistema basado en reglas. Tiene los siguientes parámetros:
  - `rule_list`: Lista de reglas con las que se creará el sistema de control
  - `fuzzy_variables`: Variables difusas extraidas de `FuzzyEncoder.fuzzy_vars`
  - `discrete_variables`: Diccionario con las variables discretas.
  - `target_variable`: Diccionario con la variable objetivo.
- `predict`: Devuelve la clasificación para una instancia de la base de datos segun el sistema basado en reglas
- `score`: Devuelve el score de la base de datos

In [84]:
from skfuzzy import control as ctrl
from abc import ABC, abstractmethod

class RuleBasedSystem(ABC):
    def __init__(self):
        self.ctrl = None

    def _create_rbs(self, rule_list, fuzzy_variables, discrete_variables, target_variable):
        
        # PASO 1
        
        # Hay que obtener todos los antecedentes. Un antecedente en 
        # scikit-fuzzy se crea con ante = ctrl.Antecedent(universe, variable_name)
        # Para ello, tanto en fuzzy_variables como en discrete_variables
        # deberia estar ya representado el universo de cada variable

        # Ademas, para cada antecedente hay que crear todos los conjuntos difusos
        # (o valores para las variables discretas) correspondientes. Tenemos las etiquetas
        # en las fuzzy_variables, y se pueden obtener tambien de las variables discretas
        # Para cada conjunto difuso se crea con ante[label] = pert, donde ya tenemos
        # las funciones de pertenencia tanto en fuzzy_variables como en discrete_variables

        antecedentes = []

        for discreteVariable in discrete_variables.keys():

            ante = ctrl.Antecedent(discrete_variables[discreteVariable]['universe'], discreteVariable)

            for dvdv in discrete_variables[discreteVariable]['pert']:
                #print("dvdv: " + str(dvdv))
                ante[dvdv] = discrete_variables[discreteVariable]['pert'][dvdv]
            
            antecedentes.append(ante)

        for fuzzyVariable in fuzzy_variables.aux_fuzzy_vars:

            ante = ctrl.Antecedent(fuzzy_variables.aux_fuzzy_vars[fuzzyVariable]['universe'], fuzzyVariable)

            for label in fuzzy_variables.labels:
                ante[label] = fuzzy_variables.aux_fuzzy_vars[fuzzyVariable]['pert'][label]

            antecedentes.append(ante)

        # PASO 2

        # Para crear el consecuente se hace con conseq = ctrl.Consequent(universe, conseq_name)
        # Por eso es mejor pasarlo por separado. De igual manera, hay que asignar cada uno 
        # de los valores que puede tomar y sus funciones de pertenencia

        conseq = ctrl.Consequent(target_variable['RainTomorrow']['universe'], 'RainTomorrow')
        conseq['No'] = target_variable['RainTomorrow']['pert'][0]
        conseq['Yes'] = target_variable['RainTomorrow']['pert'][1]

        rules = []

        # PASO 3

        # Finalmente hay que generar las reglas. Para ello se crean con 
        # rule = ctrl.Rule(antecedent=antecedents, consequent=consequent)
        # Aqui trabajamos con los antecedentes y el consecuente que hemos generado
        # anteriormente. Con la lista de reglas que tenemos del arbol, generamos
        # las reglas con el formato de scikit-fuzzy. antecedents es una union logica 
        # de terminos, esto quiere decir, ante1[label1] & ante2[label2]...
        # De igual manera, el consecuente es conseq[label] para el valor correspondiente
        # de la regla. Habra que haber almacenado los antecedentes para poder acceder
        # a ellos en funcion del nombre de la columna que aparece en la regla

        for regla in rule_list:

            antedecentes = []

            for antedecente in antecedentes:

                for a in regla[0]:

                    if antedecente.label == a[0]:
                        antedecentes.append(antedecente[a[1]])

            consecuente = None
            if regla[1] == 0:
                consecuente = conseq['No']
            else:
                consecuente = conseq['Yes']

            rule = ctrl.Rule(self.conseguirAntecendentesRecursivamente(len(antedecentes), antedecentes), consecuente)
            rules.append(rule)

        for rule in rules:
            print(rule)
            print("")

        self.ctrl = ctrl.ControlSystem(rules=rules)

    def conseguirAntecendentesRecursivamente(self, len, antecedentes):

        # Si la longitud es 1, entonces el método para y devuelve el último antecedente que queda
        if len == 1:
            return antecedentes[0]

        # Eliminamos el primer antecedente y pasamos el resto como parámetro
        aux = []
        for a in antecedentes:

            if a == antecedentes[0]:
                continue

            aux.append(a)

        # Devolvemos la operación lógica entre el antecedente y el resto de antecedentes, que se calcularán recursivamente.
        return antecedentes[0] & self.conseguirAntecendentesRecursivamente(len - 1, aux)

    @abstractmethod
    def fit():
        pass
    
    def predict(self, x):
        sim = ctrl.ControlSystemSimulation(self.ctrl, flush_after_run=1000)
        # Para predecir, hay que añadir los valores de la instancia al sistema
        # de simulacion de control. Primero hay que ver si existe el antecedente 
        # en el sistema de control (posiblemente las reglas extraidas del arbol
        # no tengan todas las caracteristicas de la instancia). En caso afirmativo,
        # la variable 'temperatura' = 8.7 se añade como sim.input['temperatura'] = 8.7    

        # TODO: IMPLEMENTAR

        # Despues se calculan los grados de las reglas
        sim.compute()

        # Finalmente se devuelve el resultado que se encuentra en sim.output['target']
        # si la variable a predecir se llama 'target'.
        
        # TODO: IMPLEMENTAR

        return ...
    

    def score(self, X, y):
        all_predictions = np.array([round(self.predict(x)) for x in X])
        return np.sum(all_predictions==y)/y.shape[0]

La clase `WangMendelRuleBasedSystem` se dividirá en las funciones:
- `_extract_rules`: encargado de procesar la bbdd. Se proporciona un formato sugerido para extraer las reglas pero se puede utilizar el formato preferido.
- `fit`: Metodo que entrena el sistema basado en reglas

In [85]:
class WangMendelRuleBasedSystem(RuleBasedSystem):
    def __init__(self):
        super().__init__()

    # Se recomienda este formato para extraer las reglas
    # del arbol, ya que simplifica la manera de tratar los 
    # antecedentes y consecuentes a la hora de generar el 
    # sistema basado en reglas
    
    def _extract_rules(self, X, y, fuzzy_variables, conjunto_difusas):

        rules = {}
        antecedentes_dict = {}

        # Para el algoritmo de Wang y Mendel, que se debe realizar en esta funcion
        # Ss recomienda el uso de un diccionario del tipo {rule: membership}
        # para que a la hora de almacenar las reglas y comprobar si hay que actualizar
        # el grado de pertenencia, sea lo mas eficiente posible.
        # Recordad que para que sea compatible con el algoritmo de la practica anterior,
        # habra que transformar ese diccionario a una lista antes de devolverlo.

        # fuzzy_variables sera un subconjunto de las variables difusas del problema, 
        # ya que un problema con tantas instancias y variables puede resultar muy 
        # complejo de extraer todas las reglas

        newX = []
        for fv in conjunto_difusas:
            newX.append(np.array(X[fv]))

        # Recorremos todas las filas de la base de datos
        for i in range(len(newX[0])):

            g = 1
            antecedentes = []
            consecuente = None

            # Recorremos todas las columnas de cada fila
            for j in range(len(newX)):
                
                # Fuzzificamos la variable
                p1 = fuzzy_variables.fuzzy_vars[conjunto_difusas[j]][0]
                p2 = fuzzy_variables.fuzzy_vars[conjunto_difusas[j]][1]
                p3 = fuzzy_variables.fuzzy_vars[conjunto_difusas[j]][2]
                valor_fuzz = fuzzificar(newX[j][i], p1, p2, p3)

                # Calculamos el antecedente
                antecedente = (conjunto_difusas[j], valor_fuzz)
                antecedentes.append(antecedente)
                
                # Calculamos el grado de recubrimiento
                g *= self.calcularGradoRecubrimiento(newX[j][i], p1, p2, p3)          

            # Calculamos el consecuente
            consecuente = y[i]

            # Creamos la regla
            regla = Regla(antecedentes, consecuente)

            # Comprobamos si la regla que acabamos de crear no está en el diccionario de reglas
            if rules.get(regla) == None:

                # Si no está, comprobamos si está el antecedente.
                if antecedentes_dict.get(tuple(antecedentes)) == None:

                    # Añadimos la nueva regla al diccionario de reglas
                    rules[regla] = g
                    antecedentes_dict[tuple(antecedentes)] = g
                else:
                    
                    # Si el antecedente está, entonces actualizamos la regla con el mayor grado de recubrimiento

                    newConsecuente = None
                    if consecuente == 0.0:
                        newConsecuente = 1.0
                    else:
                        newConsecuente == 0.0

                    rule_aux = Regla(antecedentes, newConsecuente)

                    if rules.get(rule_aux) == None:
                        continue

                        # Añadimos la nueva regla al diccionario de reglas
                        rules[rule_aux] = g
                        antecedentes_dict[tuple(antecedentes)] = g
                        
                    else:

                        g1 = g
                        g2 = antecedentes_dict[tuple(antecedentes)]

                        if g1 > g2:
                            
                            # Eliminamos la regla anterior que ya estaba en ambos diccionarios
                            # del rules[rule_aux]
                            rules.pop(rule_aux)
                            # del antecedentes_dict[tuple(antecedentes)]
                            antecedentes_dict.pop(tuple(antecedentes))
                            
                            # Añadimos la nueva regla al diccionario de reglas
                            rules[regla] = g
                            antecedentes_dict[tuple(antecedentes)] = g

            else:

                # Si está, cambiamos el grado de recubrimiento de la regla por el que sea mayor.
                g1 = rules[regla]
                g2 = g

                # Si g2 es mayor, se actualiza, si no se queda como está
                if g2 > g1:
                    rules[regla] = g2

            #if i == 5: break

        # Transformar diccionario en lista
        new_rules = []
        for r in rules:
            new_rules.append(r.getRegla())

        for r in new_rules:
            print(r)

        print("")

        return new_rules

    def calcularGradoRecubrimiento(self, valor, p1, p2, p3):

        if valor <= p1 or valor == p2 or valor >= p3:
            return 1

        if valor > p1 and valor < p2:
            pUno = p1
            pDos = p2
        else:
            pUno = p2
            pDos = p3
 
        max = pDos - pUno
        valUno = valor - pUno
        valDos = pDos - valor
        valUno = valUno / max
        valDos = valDos / max

        if valUno > valDos:
            return valUno
        else:
            return valDos

    def fit(self, X, y, fuzzy_variables, discrete_variables, target_variable, conjunto_difusas):
        rule_list = self._extract_rules(X, y, fuzzy_variables, conjunto_difusas)
        self._create_rbs(rule_list, fuzzy_variables, discrete_variables, target_variable)


In [86]:
from collections import abc

class Regla(abc.Hashable):
    def __init__(self, antecedentes, consecuente):
        self.antecedentes = antecedentes
        self.consecuente = consecuente
        self.regla = (self.getTuple(antecedentes), consecuente)
        self.codigo_hash = hash(self.regla)

    def getTuple(self, antecedentes):

        t = ()
        for ante in antecedentes:
            newAnte = (ante)
            t = t + newAnte

        return t

    def getRegla(self):
        return (self.antecedentes, self.consecuente)

    def __hash__(self):
        return self.codigo_hash

    def __eq__(self, regla):
        return self.antecedentes == regla.antecedentes and self.consecuente == regla.consecuente

    def __str__(self):
        return "(" + str(self.antecedentes) + ", " + str(self.consecuente) + ")"

---

<br>

<a id="data"></a>
# <font color="#004D7F"> 4. Resultados</font>

Para finalizar esta práctica, el alumno deberá ejecutar el algoritmo con distintos conjuntos de variables difusas, y compararlo tanto en score como en tiempo de ejecución con el algoritmo de la práctica anterior. Habrá que hacer comparativas mínimo con conjuntos de 3, 4 y 5 variables difusas.

### <font color="#004D7F"> 4.1 Prueba con 3 variables</font>
Como podemos observar, el algortimo recorre las reglas, dejando solo las de mayor recubrimiento de las 3 variables

In [87]:
conjunto_difusas = ['MinTemp', 'MaxTemp', 'Rainfall']

wmrbs = WangMendelRuleBasedSystem()
wmrbs.fit(X, y, fuzzy_vars, discrete_variables, target_variable, conjunto_difusas)

([('MinTemp', 'Mid'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'High'), ('Rainfall', 'Low')], 0.0)
([('MinTemp', 'High'), ('MaxTemp', 'High'), ('Rainfall', 'Low')], 0.0)
([('MinTemp', 'Low'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low')], 0.0)
([('MinTemp', 'Low'), ('MaxTemp', 'Low'), ('Rainfall', 'Low')], 0.0)
([('MinTemp', 'High'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'Mid'), ('Rainfall', 'Mid')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'Mid'), ('Rainfall', 'High')], 1.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'Low'), ('Rainfall', 'Low')], 0.0)
([('MinTemp', 'High'), ('MaxTemp', 'Mid'), ('Rainfall', 'Mid')], 1.0)

IF MinTemp[Mid] AND (MaxTemp[Mid] AND Rainfall[Low]) THEN RainTomorrow[No]
	AND aggregation function : fmin
	OR aggregation function  : fmax

IF MinTemp[Mid] AND (MaxTemp[High] AND Rainfall[Low]) THEN RainTomorrow[No]
	AND aggregation function : fmin
	OR aggregation function  : fmax

IF MinTemp[High] A

### <font color="#004D7F"> 4.1 Prueba con 4 variables</font>
Como podemos observar, el algortimo recorre las reglas, dejando solo las de mayor recubrimiento de las 4 variables

In [88]:
conjunto_difusas = ['MinTemp', 'MaxTemp', 'Rainfall', 'Sunshine']

wmrbs = WangMendelRuleBasedSystem()
wmrbs.fit(X, y, fuzzy_vars, discrete_variables, target_variable, conjunto_difusas)

([('MinTemp', 'Mid'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'High'), ('Rainfall', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'High'), ('MaxTemp', 'High'), ('Rainfall', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'Low'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'Low'), ('MaxTemp', 'Low'), ('Rainfall', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'High'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'Mid'), ('Rainfall', 'Mid'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'High'), ('Rainfall', 'Low'), ('Sunshine', 'High')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low'), ('Sunshine', 'High')], 0.0)
([('MinTemp', 'High'), ('MaxTemp', 'High'), ('Rainfall', 'Low'), ('Sunshine', 'Low')], 0.0)
([('MinTemp', 'High'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low'), ('Sunshine', 'High')], 0.0)

### <font color="#004D7F"> 4.1 Prueba con 5 variables</font>
Como podemos observar, el algortimo recorre las reglas, dejando solo las de mayor recubrimiento de las 5 variables

In [89]:
conjunto_difusas = ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine']

wmrbs = WangMendelRuleBasedSystem()
wmrbs.fit(X, y, fuzzy_vars, discrete_variables, target_variable, conjunto_difusas)

([('MinTemp', 'Mid'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low'), ('Evaporation', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'High'), ('Rainfall', 'Low'), ('Evaporation', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'High'), ('MaxTemp', 'High'), ('Rainfall', 'Low'), ('Evaporation', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'Low'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low'), ('Evaporation', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'Low'), ('MaxTemp', 'Low'), ('Rainfall', 'Low'), ('Evaporation', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'High'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low'), ('Evaporation', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'Mid'), ('Rainfall', 'Mid'), ('Evaporation', 'Low'), ('Sunshine', 'Mid')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'High'), ('Rainfall', 'Low'), ('Evaporation', 'Low'), ('Sunshine', 'High')], 0.0)
([('MinTemp', 'Mid'), ('MaxTemp', 'Mid'), ('Rainfall', 'Low'), ('Evaporation', 'Lo

---

<br>

<a id="data"></a>
# <font color="#004D7F"> 5. Entrega</font>

Al igual que en la práctica anterior, el alumno subirá a campus virtual un notebook con la implementación del sistema de reglas y del proceso de inferencia.

Para aprobar la práctica será necesario que la implementación del algoritmo sea correcta. Se tendrán en cuenta para la nota la eficiencia de la implementación, así como la limpieza y los comentarios en el código.

La parte de resultados será necesaria llevarla a cabo para optar a la nota máxima.

La entrega de esta práctica tiene de plazo hasta el día 5 de Junio de 2022.

### <font color="#004D7F"> Conclusiones</font>
Hemos visto tras realizar la practica, que haciendo un buen uso del algoritmo junto con técnicas de transformación de dataframes a numpy, es más interesnate y eficiente que aprender las reglas por generacion de arbol en profundidad.
De esta manera solo nos centramos en aprender las reglas mas interesantes mediante el recubrimiento, junto con las variables que nos interesan, para obtener resultados mas directos